In [ ]:
import torch
import pandas as pd
import numpy as np
import io
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from torch.optim import Adam
import torch.nn.functional as F
import random
import time
import datetime

In [2]:
dlg_list=[]
with io.open("data/textandembedding/allmorptok_inputtextfile.txt", encoding='euc-kr') as f:
    for a in f:
        b = a.strip().split('\t')
        dlg_list.append(b)

dlg_df=pd.DataFrame(dlg_list, columns=["dialog",'label'])
dlg_df["label"] = pd.to_numeric(dlg_df["label"])

In [3]:
type(dlg_df["label"][0])

numpy.int64

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [5]:
dialog_data_shuffled = dlg_df.sample(frac=1).reset_index(drop=True)
dialog_data_shuffled_token = dialog_data_shuffled.copy()
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    dialog_data_shuffled_token["dialog"][idx] = tokenizer.tokenize("[CLS] " + text + " [SEP]")

C:\Users\tlsgh\AppData\Local\Temp/ipykernel_7756/3012246228.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dialog_data_shuffled_token["dialog"][idx] = tokenizer.tokenize("[CLS] " + text + " [SEP]")


In [6]:
dialog_data_shuffled_token

,dialog,label
0,"[[CLS], 안, ##녕, 하, 세, ##요, 코, ##디, 봇, 입, ##니다,...",1
1,"[[CLS], 안, ##녕, 하, 세, ##요, 코, ##디, 봇, 입, ##니다,...",1
2,"[[CLS], 어, ##서, 오, 세, ##요, 코, ##디, 봇, 입, ##니다,...",1
3,"[[CLS], 안, ##녕, 하, 세, ##요, 코, ##디, 봇, 입, ##니다,...",1
4,"[[CLS], 안, ##녕, 하, 세, ##요, 코, ##디, 봇, 입, ##니다,...",1
...,...,...
58959,"[[CLS], 안, ##녕, 하, 세, ##요, 코, ##디, 봇, 입, ##니다,...",0
58960,"[[CLS], 어, ##서, 오, 세, ##요, 코, ##디, 봇, 입, ##니다,...",0
58961,"[[CLS], 어, ##서, 오, 세, ##요, 코, ##디, 봇, 입, ##니다,...",1
58962,"[[CLS], 안, ##녕, 하, 세, ##요, 코, ##디, 봇, 입, ##니다,...",0


In [7]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    dialog_data_shuffled_token["dialog"][idx] = dialog_data_shuffled_token["dialog"][idx][:512]

C:\Users\tlsgh\AppData\Local\Temp/ipykernel_7756/291199888.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dialog_data_shuffled_token["dialog"][idx] = dialog_data_shuffled_token["dialog"][idx][:512]


In [8]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    textlen = len(text)
    if textlen == 512:
        dialog_data_shuffled_token["dialog"][idx][511] = "[SEP]"

In [9]:
max10=0
for lt in dialog_data_shuffled_token["dialog"]:
    if max10 < len(lt):
        max10=len(lt)
print(max10)

512


In [10]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    textlen = len(text)
    if textlen == 512:
        text[-1]

In [11]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    dialog_data_shuffled_token["dialog"][idx] = tokenizer.convert_tokens_to_ids(text)

C:\Users\tlsgh\AppData\Local\Temp/ipykernel_7756/3902430955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dialog_data_shuffled_token["dialog"][idx] = tokenizer.convert_tokens_to_ids(text)


In [12]:
dialog_data_shuffled_token["dialog"][0]


[101,
 9521,
 118741,
 9952,
 9435,
 48549,
 9812,
 48446,
 9361,
 9645,
 48345,
 9294,
 119137,
 9633,
 9087,
 12638,
 9113,
 85836,
 118671,
 48549,
 8877,
 78123,
 9385,
 12965,
 9766,
 17730,
 9559,
 8847,
 9137,
 9645,
 8932,
 9130,
 118848,
 9954,
 9812,
 48446,
 9765,
 38631,
 9960,
 9689,
 9435,
 48549,
 9011,
 9655,
 14040,
 9248,
 8932,
 11903,
 26737,
 9689,
 9435,
 48549,
 9901,
 12605,
 119354,
 9875,
 9909,
 118823,
 9638,
 9928,
 12030,
 15184,
 9640,
 9924,
 11102,
 9812,
 48446,
 9645,
 48345,
 9414,
 10459,
 9164,
 9952,
 10459,
 9043,
 9246,
 32158,
 9559,
 9113,
 9043,
 28911,
 9519,
 9604,
 21876,
 9164,
 9487,
 51431,
 9637,
 9416,
 14871,
 9638,
 9246,
 32158,
 9559,
 9521,
 9117,
 9546,
 48549,
 9378,
 118855,
 9801,
 30873,
 9668,
 68984,
 9590,
 9130,
 118848,
 9954,
 9993,
 41620,
 9909,
 118823,
 9365,
 60479,
 9043,
 9546,
 118833,
 24982,
 48549,
 9319,
 14040,
 9489,
 35866,
 119142,
 9309,
 48446,
 8934,
 9638,
 78838,
 9565,
 100,
 9942,
 42769,
 15184,

In [13]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    text_len=len(text)
    dialog_data_shuffled_token["dialog"][idx] = text + [0] * (512-text_len)

C:\Users\tlsgh\AppData\Local\Temp/ipykernel_7756/3376163152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dialog_data_shuffled_token["dialog"][idx] = text + [0] * (512-text_len)


In [14]:
len(dialog_data_shuffled_token["dialog"][0])

512

In [15]:
train = dialog_data_shuffled_token[:47170]
test = dialog_data_shuffled_token[47170:]

In [16]:
train

,dialog,label
0,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
1,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
2,"[101, 9546, 12424, 9580, 9435, 48549, 9812, 48...",1
3,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
4,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
...,...,...
47165,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
47166,"[101, 9546, 12424, 9580, 9435, 48549, 9812, 48...",1
47167,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
47168,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1


In [17]:
max = 0
for text1 in train["dialog"]:
    if max < len(text1):
        max = len(text1)
print(max)
max1 = 0
for text2 in test["dialog"]:
    if max1 < len(text2):
        max1 = len(text2)
print(max1)

512
512


In [9]:
print(train["dialog"][10])
print(tokenizer.tokenize((train["dialog"][10])))
print(len(tokenizer.tokenize((train["dialog"][10]))))
print(tokenizer.covert_token_to_ids)

어서 오 세요 코디 봇 입니다 무엇 을 도와 드릴까요 사촌 오빠 결혼식 에 참석 하 려는 데 입 고 갈 하객 룩 추천 해 주 세요 치마 바지 원피스 중 어떤 옷 이 포함 된 코디 로 추천 해 드릴까요 치마 가 포함 된 코디 요 선호 하 는 색상 이나 스타일 이 있 으신 가요 베이지 색상 을 선호 해요 잠시 만 기다려 주 세요 하객 룩 에 적절 한 단정 한 느낌 의 A 라인 스커트 가 포함 된 코디 입니다 요청 하 신 색상 에 맞춰 코디 해 보 았 습니다 마음 에 드 시 나요 아우 터만 다른 스타일 로 추천 해 주 세요 나머지 는 마음 에 들 어요 선호 하 시 는 색상 이나 스타일 이 있 으신 가요 깔끔 한 스타일 의 검은색 코트 였 으면 좋 겠 어요 잠시 만 기다려 주 세요 스트레이트 실루엣 으로 단정 한 느낌 을 주 며 울 혼방 으로 겨울 에 입 기 적절 한 코트 입니다 스트레이트 실루엣 미디 길 이 더블 여 밈 벨트 여 밈 테일 러드 칼라 드 랍 숄더 일자 형 소매 손등 덮 는 길 이 허리선 양쪽 패치 포켓 울 무광 불 투명 신축성 없 는 두꺼운 블랙 음악회 차분 한 여유 있 는 남녀 공용 의 단순 한 겨울 느낌 의
['어', '##서', '오', '세', '##요', '코', '##디', '봇', '입', '##니다', '무', '##엇', '을', '도', '##와', '드', '##릴', '##까', '##요', '사', '##촌', '오', '##빠', '결', '##혼', '##식', '에', '참', '##석', '하', '려', '##는', '데', '입', '고', '갈', '하', '##객', '룩', '추', '##천', '해', '주', '세', '##요', '치', '##마', '바', '##지', '원', '##피', '##스', '중', '어떤', '옷', '이', '포', '##함', '된', '코', '##디', '로', '추', '##천', '해', '드', '##릴', '##까', '##요', '치', '##마', '가', 

In [18]:
class FashionDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]
        return text, label

In [26]:
train_dataset = FashionDataset(train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

In [25]:
for text,label in train_loader:
    print(text,label)

[tensor([101]), tensor([9521]), tensor([118741]), tensor([9952]), tensor([9435]), tensor([48549]), tensor([9812]), tensor([48446]), tensor([9361]), tensor([9645]), tensor([48345]), tensor([9294]), tensor([119137]), tensor([9633]), tensor([9087]), tensor([12638]), tensor([9113]), tensor([85836]), tensor([118671]), tensor([48549]), tensor([8843]), tensor([52560]), tensor([9117]), tensor([8898]), tensor([9477]), tensor([88332]), tensor([41620]), tensor([13890]), tensor([9559]), tensor([9025]), tensor([9176]), tensor([8849]), tensor([48345]), tensor([9588]), tensor([9633]), tensor([9765]), tensor([38631]), tensor([9960]), tensor([9689]), tensor([9435]), tensor([48549]), tensor([9011]), tensor([9655]), tensor([14040]), tensor([9248]), tensor([8932]), tensor([11903]), tensor([26737]), tensor([9689]), tensor([9435]), tensor([48549]), tensor([9130]), tensor([118848]), tensor([9954]), tensor([9260]), tensor([75855]), tensor([118916]), tensor([9638]), tensor([9928]), tensor([48533]), tensor([909

KeyboardInterrupt: 

In [23]:
device = torch.device("cuda")
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [24]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0
MAX_LEN = 512 #최대 시퀀스 길이 설정
model.train()

for epoch in range(epochs):

    for text, label in train_loader:
        optimizer.zero_grad()

        # encoding and zero padding

        sample = torch.tensor(text)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

C:\Users\tlsgh\AppData\Local\Temp/ipykernel_7756/2571588945.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [17]:
torch.save(model,"./data/bert1.pt")


In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):

    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_loader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")